In [1]:
#import necessary packages:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder

In [2]:
with open('PostalCodesofCanada.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
match = soup.title
print(match)

data = []
table = soup.find('table', attrs={'class':'lineItemsTable'})
table_body = soup.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

<title>List of postal codes of Canada: M - Wikipedia</title>


In [3]:
postal_codes = pd.DataFrame(data)
postal_codes.columns = ['PostalCode', 'Borough', 'Neighbourhood'] 
indexNames = postal_codes[postal_codes['Borough'] == 'Not assigned'].index
postal_codes.drop(indexNames , inplace=True)
postal_codes = postal_codes.groupby(['PostalCode', 'Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
postal_codes = postal_codes.reset_index()
indexes = postal_codes[postal_codes['Neighbourhood'] == 'Not assigned'].index
for i in indexes:
    postal_codes.at[i, 'Neighbourhood'] = postal_codes.at[i, 'Borough']

In [4]:
postal_codes = postal_codes.set_index('PostalCode')
postal_codes.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [21]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords = coords.set_index('Postal Code')
coords.head()


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [25]:
PC = postal_codes.join(coords, on='PostalCode')
PC.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
CLIENT_ID = 'MKI02ZEY4WWFUJ4ULFFV3NRZO3TW54A5FLS3HRVJ0B3A5NN5' # your Foursquare ID
CLIENT_SECRET = 'LL01H43FDP1YBDVY5LSGJBHM02BFYXHF0T4M0WSBF2ZCQEFR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
neighborhood_latitude = PC.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = PC.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = PC.loc[0, 'Neighbourhood']

Your credentails:
CLIENT_ID: MKI02ZEY4WWFUJ4ULFFV3NRZO3TW54A5FLS3HRVJ0B3A5NN5
CLIENT_SECRET:LL01H43FDP1YBDVY5LSGJBHM02BFYXHF0T4M0WSBF2ZCQEFR


In [18]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?search?client_id=MKI02ZEY4WWFUJ4ULFFV3NRZO3TW54A5FLS3HRVJ0B3A5NN5&client_secret=LL01H43FDP1YBDVY5LSGJBHM02BFYXHF0T4M0WSBF2ZCQEFR&ll=nan,nan&v=20180605&radius=500&limit=100'

In [19]:
neighborhood_latitude = 43.806686
neighborhood_longitude = -79.194353
url = 'https://api.foursquare.com/v2/venues/explore?search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)


{'meta': {'code': 400,
  'errorType': 'invalid_auth',
  'errorDetail': 'Missing access credentials. See https://developer.foursquare.com/docs/api/configuration/authentication for details.',
  'requestId': '5d459db1d69ed00032c215ec'},
 'response': {}}

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize

The code gives us a lot of trouble parsing trough the son file so we're going to tear it down:

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
Tor_ven = getNearbyVenues(names=PC['Neighbourhood'],
                                   latitudes=PC['Latitude'],
                                   longitudes=PC['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [28]:
Tor_ven.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [29]:
Tor_ven.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55


In [30]:
# one hot encoding
t = pd.get_dummies(Tor_ven[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t['Neighborhood'] = Tor_ven['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t.columns[-1]] + list(t.columns[:-1])
t = t[fixed_columns]

t.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
t_group = t.groupby('Neighborhood').mean().reset_index()
t_group.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [32]:
num_top_venues = 5

for hood in t_group['Neighborhood']:
    print("----"+hood+"----")
    temp = t_group[t_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Agincourt----
                venue  freq
0      Clothing Store   0.2
1              Lounge   0.2
2  Chinese Restaurant   0.2
3      Sandwich Place   0.2
4      Breakfast Spot   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1                       Park  0.33
2                Coffee Shop  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store   0.2
1           Pizza Place   0.1
2           Coffee Shop   0.1
3        Sandwich Place   0.1
4  Fast Food Restaurant   0.1


----Alderwood, Long Branch----
            venue  

               venue  freq
0     Discount Store  0.29
1         Hobby Shop  0.14
2  Convenience Store  0.14
3   Department Store  0.14
4        Coffee Shop  0.14


----East Toronto----
                       venue  freq
0                       Park   0.4
1                Pizza Place   0.2
2                Coffee Shop   0.2
3          Convenience Store   0.2
4  Middle Eastern Restaurant   0.0


----Emery, Humberlea----
                    venue  freq
0  Furniture / Home Store   0.5
1          Baseball Field   0.5
2             Yoga Studio   0.0
3        Malay Restaurant   0.0
4     Monument / Landmark   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.11
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3   Japanese Restaurant  0.03
4            Food Court  0.03


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2        Hotel  0.04
3   Steakhouse  0.04
4   Restauran

                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.06
2             Cosmetics Shop  0.04
3  Middle Eastern Restaurant  0.03
4       Fast Food Restaurant  0.03


----Scarborough Village----
                 venue  freq
0           Playground   0.5
1           Smoke Shop   0.5
2          Yoga Studio   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Silver Hills, York Mills----
                       venue  freq
0                  Cafeteria   0.5
1          Martial Arts Dojo   0.5
2  Middle Eastern Restaurant   0.0
3                      Motel   0.0
4        Monument / Landmark   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.05
2          Restaurant  0.05
3                Café  0.05
4               Hotel  0.04


----Stn A PO Boxes 25 The Esplanade----
                  venue  freq
0           Coffee Shop  0.11
1                  Café  0.04
2            Res

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =  num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = t_group['Neighborhood']

for ind in np.arange(t_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()['Neighborhood']

for ind in np.arange(t_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Gym,American Restaurant,Breakfast Spot,Restaurant,Burger Joint
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Clothing Store,Women's Store,Diner,Discount Store,Dive Bar,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Fried Chicken Joint,Video Store,Pharmacy,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Dive Bar
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Gym,Airport Service,Department Store


In [35]:
from sklearn.cluster import KMeans
kclusters = 5

ts = t_group.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ts)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [40]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_, allow_duplicates=True)
t_merged = PC


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
t_merged = t_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

t_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Cluster Labels,Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,1.0,1.0,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,0.0,0.0,0.0,Bar,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,0.0,0.0,0.0,Mexican Restaurant,Electronics Store,Pizza Place,Breakfast Spot,Intersection,Rental Car Location,Medical Center,Women's Store,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,0.0,0.0,0.0,Coffee Shop,Insurance Office,Korean Restaurant,Mexican Restaurant,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Diner,Dive Bar,Dog Run


In [45]:
t_merged.columns = ['PostalCode',
 'Borough',
 'Neighbourhood',
 'Latitude',
 'Longitude',
 'Clusters',
 'Cluster Labels',
 'Cluster Labels',
 'Cluster Labels',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

t_merged.drop('Cluster Labels', axis=1)
t_merged = t_merged.sort_values('Clusters')[:100]
t_merged = t_merged.reset_index()
t_merged.head()
t_merged = t_merged.drop('index', axis=1)

In [55]:
t_merged.head()

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Clusters,Cluster Labels,Cluster Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0.0,0.0,0.0,0.0,Coffee Shop,Pub,Pizza Place,Restaurant,Café,Market,Bakery,Italian Restaurant,Chinese Restaurant,Playground
1,68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.0,0.0,0.0,0.0,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Airport Gate
2,67,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,0.0,0.0,0.0,0.0,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Bar,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Bakery,Farmers Market
3,66,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,0.0,0.0,0.0,0.0,Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Bakery,Coffee Shop,Chinese Restaurant,Poutine Place,Pub
4,65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0.0,0.0,0.0,0.0,Sandwich Place,Coffee Shop,Café,Pizza Place,Cosmetics Shop,American Restaurant,Jewish Restaurant,Pub,Furniture / Home Store,Indian Restaurant
